## Домашняя работа 12 - Продвинутый pandas
### Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений

In [60]:
import pandas as pd

In [61]:
log = pd.read_csv('.\\HW-12 data\\visit_log.csv', sep=';')

In [62]:
def get_source_type(row):
    if row['traffic_source'] in ['yandex','google']: 
        return 'organic'
    elif row['traffic_source'] in ['paid','email']: 
        if row['region'] == 'Russia': return 'ad'
        else: return 'other'
    else: return row['traffic_source']

In [63]:
%%time

log['source_type'] = log.apply(get_source_type, axis=1) 
log.head()

Wall time: 432 ms


,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [65]:
%%time

log.loc[log.traffic_source.isin(['yandex','google']),'source_type2'] = 'organic'
log.loc[log.traffic_source.isin(['paid','email'])&log.region.str.contains('Russia'),'source_type2'] = 'ad'
log.loc[log.traffic_source.isin(['paid','email'])&(~log.region.isin(['Russia'])),'source_type2'] = 'other'
log.loc[~log.traffic_source.isin(['yandex','google','paid','email']),'source_type2'] = log.traffic_source
log.head(5)


Wall time: 15.6 ms


,timestamp,visit_id,url,region,user_id,traffic_source,source_type,source_type2
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic,organic


In [68]:
log.loc[log.source_type!=log.source_type2] 

,timestamp,visit_id,url,region,user_id,traffic_source,source_type,source_type2


### Задание 2
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

- Прочитайте содержимое файла с датафрейм
- Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном


In [124]:
URLs = pd.read_csv('.\\HW-12 data\\URLs.txt',sep='/')

ParserError: Error tokenizing data. C error: Expected 3 fields in line 5, saw 4


In [132]:
import re
def get_region(string):
    str_list = re.findall(r'\w+', string)
    for word in str_list:
        if word.lower() in ['москва', 'тула', 'ярославль']:
            return 'Центр'
        elif word.lower() in ['петербург', 'псков', 'мурманск']:
            return 'Северо-Запад'
        elif word.lower() in ['владивосток', 'сахалин', 'хабаровск']:
            return 'Дальний Восток'
    return 'undefined'

In [133]:
keywords['region'] = keywords['keyword'].apply(get_region)
check = keywords[keywords['region'] != 'undefined']
check.head(10)

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад
1259,москва,153050,Центр
1346,санкт петербург,144232,Северо-Запад
1396,гисметео москва,140279,Центр
1482,госуслуги москва,133061,Центр
1579,авито тула,126168,Центр


### Задание 3
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [79]:
ratings = pd.read_csv('.\\HW-12 data\\ml-latest-small\\ratings.csv.')
#ratings = pd.read_csv('.\\HW-11 data\\ml-latest-small\\ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [124]:
def production_year(string):
    year =0
    try:
        year = re.search(r'(?!\()\d{4}(?=\))',string).group(0)
    except: pass
    if int(year) in range(1950,2011):
        return int(year)
    else:
        return 1900

In [119]:
ratings_agg = (ratings[['userId','rating','timestamp']]
        .groupby(['userId'])
        .agg({'rating': {'count'},'timestamp': {'max','min'}})
        )
ratings_agg.head()




rating   timestamp            
        count         min         max
userId                               
1          20  1260759108  1260759205
2          76   835355395   835356246
3          51  1298861589  1298932787
4         204   949778714   949982274
5         100  1163373044  1163375145

In [ ]:
ratings_filt = ratings_agg[(ratings_agg['rating']['count']>100)]

ratings_filt['LifeTime'] = ratings_filt['timestamp']['max']-ratings_filt['timestamp']['max']
log.loc[log.traffic_source.isin(['paid','email'])&log.region.str.contains('Russia'),'source_type2'] = 'ad'

In [126]:
result = (data[['year','rating']].groupby(['year']).mean()
 .sort_values('rating',ascending=False)
)
result.head(10)

,rating
year,
1957,4.014241
1972,4.011136
1952,4.000000
1954,3.992754
1951,3.983539
1974,3.978625
1962,3.952316
1950,3.915254
1977,3.905506


### Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).
Необходимо сформировать две таблицы:

- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.